Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Train in a remote VM (MLC managed DSVM)
* Create Workspace
* Create Experiment
* Upload data to a blob in workspace
* Configure ACI run config
* Submit the experiment in ACI
* Register the retrained model

# Prerequisites
Make sure you go through the [00. Installation and Configuration](00.configuration.ipynb) Notebook first if you haven't.

## Install Azure ML SDK

* !pip install azureml-core
* !pip install azureml-contrib-iot
* !pip install azure-mgmt-containerregistry

## Check the conda environment
Make sure you have started the notebook from the correct conda environment

In [1]:
import os
print(os.__file__)

/home/arun/anaconda3/envs/tf_dev/lib/python3.5/os.py


In [2]:
# Check core SDK version number
import azureml.core as azcore

print("SDK version:", azcore.VERSION)

SDK version: 1.0.2


## Initialize Workspace

Initialize a workspace object from persisted configuration.

In [3]:
from azureml.core import Workspace

ws = Workspace.from_config('./aml_config/config.json')
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

Found the config file in: /home/arun/Documents/tensorflow-for-poets-2/aml_config/config.json
peabody
peabody
eastus
54646fde-e2bd-4f13-bb8a-2eb1174d1240


## Create Experiment

**Experiment** is a logical container in an Azure ML Workspace. It hosts run records which can include run metrics and output artifacts from your experiments.

In [4]:
experiment_name = 'cats_dogs'

from azureml.core import Experiment
exp = Experiment(workspace = ws, name = experiment_name)

## Upload data files into datastore
Register your existing azure storage as a new datastore with the workspace. The datastore should be backed by the Azure blob storage account. We can use it to transfer data from local to the cloud, and access it from the compute target.

In [5]:
from azureml.core.datastore import Datastore
ds = Datastore.register_azure_blob_container(workspace=ws, 
                                         datastore_name='mycatdog', 
                                         container_name='cat-dog',
                                         account_name='mytraindata', 
                                         account_key='TPYHA0FQYymwr0it/Vubn/aAC8hYcuGNrp6TmicH9JidTI1PnwYeL9DZ51UnF5xN8oW26+eAWUnQOLkURa++Ig==',
                                         create_if_not_exists=False)
data_path = "training_images" # This is the path to the folder in the blob container. Set this to None to get all the contents.
print(ds.name, ds.datastore_type, ds.account_name, ds.container_name)

mycatdog AzureBlob mytraindata cat-dog


## Configure for using ACI
Linux-based ACI is available in `West US`, `East US`, `West Europe`, `North Europe`, `West US 2`, `Southeast Asia`, `Australia East`, `East US 2`, and `Central US` regions. See details [here](https://docs.microsoft.com/en-us/azure/container-instances/container-instances-quotas#region-availability).

Create a `DataReferenceConfiguration` object to inform the system what data folder to download to the copmute target.

In [6]:
from azureml.core.runconfig import DataReferenceConfiguration
dr = DataReferenceConfiguration(datastore_name=ds.name, 
                   path_on_datastore=data_path, 
                   mode='download', # download files from datastore to compute target
                   overwrite=True)

Set the system to build a conda environment based on the run configuration. Once the environment is built, and if you don't change your dependencies, it will be reused in subsequent runs.

In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "cpucluster3"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D3', max_nodes=2)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# Use the 'status' property to get a detailed status for the current AmlCompute. 
print(compute_target.status.serialize())


Found existing compute target.
{'targetNodeCount': 0, 'allocationState': 'Steady', 'currentNodeCount': 0, 'nodeStateCounts': {'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'preparingNodeCount': 0, 'preemptedNodeCount': 0, 'leavingNodeCount': 0}, 'allocationStateTransitionTime': '2019-01-11T01:06:29.709000+00:00', 'provisioningState': 'Succeeded', 'modifiedTime': '2018-12-19T01:21:19.162967+00:00', 'vmSize': 'STANDARD_D3', 'scaleSettings': {'nodeIdleTimeBeforeScaleDown': 'PT120S', 'maxNodeCount': 2, 'minNodeCount': 0}, 'creationTime': '2018-12-19T01:20:00.920301+00:00', 'provisioningStateTransitionTime': None, 'errors': None, 'vmPriority': 'Dedicated'}


In [8]:
from azureml.core.runconfig import RunConfiguration, DEFAULT_CPU_IMAGE
from azureml.core.conda_dependencies import CondaDependencies

# create a new runconfig object
run_config = RunConfiguration(framework = "python")

# Set compute target
run_config.target = compute_target.name

# set the data reference of the run configuration
run_config.data_references = {ds.name: dr}

# enable Docker 
run_config.environment.docker.enabled = True

# set Docker base image to the default CPU-based image
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE

# use conda_dependencies.yml to create a conda environment in the Docker image for execution
run_config.environment.python.user_managed_dependencies = False

# auto-prepare the Docker image when used for execution (if it is not already prepared)
run_config.auto_prepare_environment = True

# specify CondaDependencies obj
run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['tensorflow==1.8.0'])

### Submit the Experiment
Submit script to run in the Docker image in the remote VM. If you run this for the first time, the system will download the base image, layer in packages specified in the `conda_dependencies.yml` file on top of the base image, create a container and then execute the script in the container.

In [9]:
from azureml.core import Run
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory = './scripts', script = 'retrain.py', run_config = run_config, 
                      # pass the datastore reference as a parameter to the training script
                      arguments=['--image_dir', str(ds.as_download()),
                                 '--architecture', 'mobilenet_1.0_224',
                                 '--output_graph', 'outputs/retrained_graph.pb',
                                 '--output_labels', 'outputs/output_labels.txt',
                                 '--model_download_url', 'https://raw.githubusercontent.com/rakelkar/models/master/model_output/',
                                 '--model_file_name', 'imagenet_2_frozen.pb'
                                ])
run = exp.submit(config=src)

### View run history details

In [10]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
cats_dogs,cats_dogs_1547169681085,azureml.scriptrun,Queued,Link to Azure Portal,Link to Documentation


In [ ]:
run.wait_for_completion(show_output=True)

RunId: cats_dogs_1547169681085

Streaming azureml-logs/60_control_log.txt

Streaming log file azureml-logs/60_control_log.txt
Streaming log file azureml-logs/80_driver_log.txt

Streaming azureml-logs/80_driver_log.txt

https://raw.githubusercontent.com/rakelkar/models/master/model_output/mobilenet_v1_1.0_224_frozen.tgz
/tmp/imagenet
/tmp/imagenet/mobilenet_v1_1.0_224_frozen.tgz

>> Downloading mobilenet_v1_1.0_224_frozen.tgz 0.0%
>> Downloading mobilenet_v1_1.0_224_frozen.tgz 0.1%
>> Downloading mobilenet_v1_1.0_224_frozen.tgz 0.1%
>> Downloading mobilenet_v1_1.0_224_frozen.tgz 0.2%
>> Downloading mobilenet_v1_1.0_224_frozen.tgz 0.2%
>> Downloading mobilenet_v1_1.0_224_frozen.tgz 0.3%
>> Downloading mobilenet_v1_1.0_224_frozen.tgz 0.3%
>> Downloading mobilenet_v1_1.0_224_frozen.tgz 0.4%
>> Downloading mobilenet_v1_1.0_224_frozen.tgz 0.4%
>> Downloading mobilenet_v1_1.0_224_frozen.tgz 0.5%
>> Downloading mobilenet_v1_1.0_224_frozen.tgz 0.5%
>> Downloading mobilenet_v1_1.0_224_frozen.tgz

>> Downloading mobilenet_v1_1.0_224_frozen.tgz 60.6%
>> Downloading mobilenet_v1_1.0_224_frozen.tgz 60.6%
>> Downloading mobilenet_v1_1.0_224_frozen.tgz 60.7%
>> Downloading mobilenet_v1_1.0_224_frozen.tgz 60.7%
>> Downloading mobilenet_v1_1.0_224_frozen.tgz 60.8%
>> Downloading mobilenet_v1_1.0_224_frozen.tgz 60.8%
>> Downloading mobilenet_v1_1.0_224_frozen.tgz 60.9%
>> Downloading mobilenet_v1_1.0_224_frozen.tgz 60.9%
>> Downloading mobilenet_v1_1.0_224_frozen.tgz 61.0%
>> Downloading mobilenet_v1_1.0_224_frozen.tgz 61.0%
>> Downloading mobilenet_v1_1.0_224_frozen.tgz 61.1%
>> Downloading mobilenet_v1_1.0_224_frozen.tgz 61.1%
>> Downloading mobilenet_v1_1.0_224_frozen.tgz 61.2%
>> Downloading mobilenet_v1_1.0_224_frozen.tgz 61.2%
>> Downloading mobilenet_v1_1.0_224_frozen.tgz 61.3%
>> Downloading mobilenet_v1_1.0_224_frozen.tgz 61.3%
>> Downloading mobilenet_v1_1.0_224_frozen.tgz 61.4%
>> Downloading mobilenet_v1_1.0_224_frozen.tgz 61.4%
>> Downloading mobilenet_v1_1.0_224_frozen.tgz

INFO:tensorflow:Looking for images in 'Dog'
INFO:tensorflow:Looking for images in 'World'
2019-01-11 01:34:19.086904: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/1010.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/10100.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/10101.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/10103.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/10105.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/10106.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/10107.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/10108.jpg_mobilenet_1.0_224.txt


INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/102.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/1020.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/10200.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/10201.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/10202.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/10203.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:100 bottleneck files created.
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/10204.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/10205.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/10206.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/10208.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bot

INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11544.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11545.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11546.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11547.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11548.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11549.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/1155.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11550.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11551.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11552.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11553.jpg_mobilenet_1.0_224.txt


INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11647.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11648.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/1165.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11650.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11651.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11652.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11653.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11654.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11655.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11656.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11657.jpg_mobilenet_1.0_224.txt


INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11750.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11751.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11752.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11753.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11754.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11755.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11756.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11757.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11758.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/1176.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11760.jpg_mobilenet_1.0_224.txt


INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11847.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11848.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11849.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/1185.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11851.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11852.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11853.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11854.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11855.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11856.jpg_mobilenet_1.0_224.txt
INFO:tensorflow:Creating bottleneck at /tmp/bottleneck/Cat/11857.jpg_mobilenet_1.0_224.txt


### Register the Model

In [ ]:
from azureml.core.model import Model

model = run.register_model(model_name = experiment_name, model_path = 'outputs/')
print(model.name, model.url, model.version, model.id, model.created_time)

## Convert Model

In [ ]:
from azureml.contrib.iot.model_converters import SnpeConverter

# submit a compile request
compile_request = SnpeConverter.convert_tf_model(
    ws,
    source_model=model,
    input_node="input",
    input_dims="1,224,224,3",
    outputs_nodes = ["final_result"],
    allow_unconsumed_nodes = True)
print(compile_request._operation_id)

In [ ]:
# wait for the request to complete
compile_request.wait_for_completion(show_output=True)

In [ ]:
# get the compiled model
compiled_model = compile_request.result
print(compiled_model.name, compiled_model.url, compiled_model.version, compiled_model.id, compiled_model.created_time)

In [ ]:
compiled_model.download(target_dir="./converted/", exist_ok=True)

### Create Docker Image

### Show the sample application file

In [ ]:
with open('./main.py', 'r') as f:
    print(f.read())


In [ ]:
from azureml.core.image import Image
from azureml.contrib.iot import IotContainerImage

image_config = IotContainerImage.image_configuration(
                                 architecture="arm32v7",
                                 execution_script="main.py", 
                                 dependencies=["cameraapi.py","iot.py","ipcprovider.py","utility.py"],
                                 docker_file="Dockerfile",
                                 tags = ["mobilenet"],
                                 description = "MobileNet based demo module")
image = Image.create(name = "peabodymobilenet",
                     # this is the model object 
                     models = [compiled_model],
                     image_config = image_config, 
                     workspace = ws)

In [ ]:
image.wait_for_creation(show_output = True)

### Enter your container registry credentials

#### List the image to get URI

In [ ]:
container_reg = ws.get_details()["containerRegistry"]
reg_name=container_reg.split("/")[-1]
resource_group_name = ws.resource_group
container_url = "\"" + image.image_location + "\","
subscription_id = ws.subscription_id
print('{}'.format(image.image_location))
print('{}'.format(reg_name))
print('{}'.format(subscription_id))

In [ ]:
from azure.mgmt.containerregistry import ContainerRegistryManagementClient
from azure.mgmt import containerregistry
client = ContainerRegistryManagementClient(ws._auth,subscription_id)
result= client.registries.list_credentials(resource_group_name, reg_name, custom_headers=None, raw=False)
username = result.username
password = result.passwords[0].value

### Build your Deployment.json file

In [ ]:
%%writefile ./deploymentpb.json

{
  "modulesContent": {
    "$edgeAgent": {
      "properties.desired": {
        "schemaVersion": "1.0",
        "runtime": {
          "type": "docker",
          "settings": {
            "minDockerVersion": "v1.25",
            "loggingOptions": "",
            "registryCredentials": {
                

In [ ]:
#Automatically adding your acr details
acr_details = "\"" + reg_name +"\": {\n\t\t\t\"username\": \""+ username + "\",\n\t\t\t" + "\"password\":\"" + password + "\",\n\t\t\t" + "\"address\":\"" + reg_name + ".azurecr.io\"" + ",\n\t\t}"
print('{}'.format(acr_details))
%store acr_details >> deploymentpb.json

In [ ]:
%%writefile -a ./deploymentpb.json
            }
          }
        },
        "systemModules": {
          "edgeAgent": {
            "type": "docker",
            "settings": {
              "image": "mcr.microsoft.com/azureiotedge-agent:1.0",
              "createOptions": "{}",
              "env": {
                "UpstreamProtocol": {
                  "value": "MQTT"
                }
              }
            }
          },
          "edgeHub": {
            "type": "docker",
            "status": "running",
            "restartPolicy": "always",
            "settings": {
              "image": "mcr.microsoft.com/azureiotedge-hub:1.0",
              "createOptions": "{\"User\":\"root\",\"HostConfig\":{\"PortBindings\":{\"5671/tcp\":[{\"HostPort\":\"5671\"}], \"8883/tcp\":[{\"HostPort\":\"8883\"}],\"443/tcp\":[{\"HostPort\":\"443\"}]}}}",
              "env": {
                "UpstreamProtocol": {
                  "value": "MQTT "
                }
              }
            }
          }
        },
        "modules": {
          "VisionSampleModule": {
            "version": "1.0",
            "type": "docker",
            "status": "running",
            "restartPolicy": "always",
            "settings": {
              "image": 

In [ ]:
#adding your container URL
%store container_url >> deploymentpb.json

In [ ]:
%%writefile -a ./deploymentpb.json
              "createOptions": "{\"HostConfig\":{\"Binds\":[\"/data/misc/camera:/app/vam_model_folder\"],\"NetworkMode\":\"host\"},\"NetworkingConfig\":{\"EndpointsConfig\":{\"host\":{}}}}"
            }
          }
        }
      }
    },
    "$edgeHub": {
      "properties.desired": {
        "schemaVersion": "1.0",
        "routes": {
          "route": "FROM /messages/* INTO $upstream"
        },
        "storeAndForwardConfiguration": {
          "timeToLiveSecs": 7200
        }
      }
    }
  }
}

## Deploy image as an IoT module

### Set subscription to the same as your workspace

In [ ]:
%%writefile ./setsub
az account set --subscription 

In [ ]:
iot_sub=ws.subscription_id
%store iot_sub >> setsub
!sh setsub 
print ('{}'.format(iot_sub))

### Provision Azure IoT Hub

In [ ]:
#RG and location to create hub
iot_rg="vaidk_"+resource_group_name
iot_location=ws.get_details()["location"]
#temp to delete
iot_location="eastus2"
iot_hub_name="iothub-"+ ws.get_details()["name"]
iot_device_id="vadik_"+ ws.get_details()["name"]
iot_deployment_id="dpl"+ "cstmvaidk"
print('{}'.format(iot_hub_name))

In [ ]:
%%writefile ./create
#Command to create hub and device


In [ ]:
# Adding Intialization steps
regcommand="\n echo Installing Extension ... \naz extension add --name azure-cli-iot-ext \n"+ "\n echo CREATING RG "+iot_rg+"... \naz group create --name "+ iot_rg +" --location "+ iot_location+ "\n" +"\n echo CREATING HUB "+iot_hub_name+"... \naz iot hub create --name "+ iot_hub_name + " --resource-group "+ iot_rg +" --sku S1"
#print('{}'.format(regcommand))
%store regcommand >> create

### Create Identity for your device 

In [ ]:
#Adding Device ID 
create_device="\n echo CREATING DEVICE ID "+iot_device_id+"... \n az iot hub device-identity create --device-id "+ iot_device_id + " --hub-name " +  iot_hub_name +" --edge-enabled"
#print('{}'.format(create_device))
%store create_device >> create

In [ ]:
#Create command and vonfigure device 
!sh create

### Create Deployment

In [ ]:
%%writefile ./deploy
#Command to create hub and device


In [ ]:
#Add deployment command
deploy_device="\necho DELETING "+iot_deployment_id+" ... \naz iot edge deployment delete --deployment-id \"" + iot_deployment_id +"\" --hub-name \"" +  iot_hub_name +"\"\necho DEPLOYING "+iot_deployment_id+" ... \naz iot edge deployment create --deployment-id \"" + iot_deployment_id + "\" --content \"deploymentpb.json\" --hub-name \"" +  iot_hub_name +"\" --target-condition \"deviceId='"+iot_device_id+"'\" --priority 1"
print('{}'.format(deploy_device))
%store deploy_device >> deploy

In [ ]:
#run deployment to stage all work for when the model is ready 
!sh deploy

### Use this conenction string on your camera to Initialize it

In [ ]:
%%writefile ./showdetails
#Command to create hub and device

In [ ]:
#Add deployment command
get_string="\n echo THIS IS YOUR CONNECTION STRING ... \naz iot hub device-identity show-connection-string --device-id  \"" + iot_device_id + "\" --hub-name \"" +  iot_hub_name+"\""
#print('{}'.format(get_string))
%store get_string >> showdetails
!sh showdetails